In [44]:
import numpy as np
import math

# Puma Configuration
Implementation of Inverse and Forward Kinematics

In [45]:
class puma():
  def __init__(self):
    self.x=0
    self.y=0
    self.z=0

    self.a1=0.25
    self.a2=0.25
    self.a3=0.25

    self.theta1=0
    self.theta2=0
    self.theta3=0

  def setpos(self,arr):
    self.x=arr[0]
    self.y=arr[1]
    self.z=arr[2]
    self.invkine()
    
# Inverse Kinematics
  def invkine(self):
    self.theta1=np.arctan2(self.y,self.x)

    r=math.pow(math.pow(self.x,2)+math.pow(self.y,2),0.5)
    s=self.z-self.a1

    D= (math.pow(r,2)+math.pow(s,2)-math.pow(self.a2,2)-math.pow(self.a3,2)) / (2*self.a2*self.a3)
    if D>1 or D<-1 :
      print("out of the workspace")

    self.theta3 = np.arctan2((-math.pow(1-math.pow(D,2),0.5)),D)
    
    self.theta2 = np.arctan2(s,r)-np.arctan2(self.a3*np.sin(self.theta3),(self.a2+self.a3*np.cos(self.theta3)))

  def dhpara(self):
    mat = []
    mat.append([0,0,self.a1,self.theta1])
    mat.append([self.a2,np.pi/2,0,self.theta2])
    mat.append([self.a3,0,0,self.theta3])

    return mat

  def dh(self,arr):
    a=arr[0]
    alpha=arr[1]
    d=arr[2]
    theta = arr[3]

    mat = []
    b=[np.cos(theta), -np.sin(theta)*np.cos(alpha), np.sin(theta)*np.sin(alpha), a*np.cos(theta)]
    mat.append(b)

    b=[np.sin(theta), np.cos(theta)*np.cos(alpha), -np.cos(theta)*np.sin(alpha), a*np.sin(theta)]
    mat.append(b)

    b=[0, np.sin(alpha), np.cos(alpha), d]
    mat.append(b)

    b=[0,0,0,1]
    mat.append(b)

    return np.matrix(mat)

# Forward Kinematics
  # def forward(self):
  #   matrix=self.dhpara()
  #   T = np.identity(4)
  #   for i in range(len(matrix)):
  #     T=np.matmul(T,self.dh(matrix[i]))
  #   return T[:3,3].T

  def forward(self):
    z1=self.a2*np.sin(self.theta2)+self.a3*np.sin(self.theta2+self.theta3)+self.a1

    r=self.a2*np.cos(self.theta2)+self.a3*np.cos(self.theta2+self.theta3)
    x1=r*np.cos(self.theta1)
    y1=r*np.sin(self.theta1)

    return [x1,y1,z1]

  def verify(self,arr):
    self.setpos(arr)
    print(f"Original Co-ordinates:- {[self.x,self.y,self.z]}\n")
    print(f"Verified from code :- {self.forward()}\n")
    print(f"Joint parameters: \n theta1: {self.theta1} rad \n theta2: {self.theta2} rad \n theta3: {self.theta3} rad \n\n")

# Stanford Configuration
Implementation of Inverse and Forward Kinematics

In [46]:
class stanford():
  def __init__(self):
    self.x=0
    self.y=0
    self.z=0

    self.theta1=0
    self.theta2=0
    self.d3=0

    self.a1=0.25
    self.a2=0.25

  def setpos(self,arr):
    self.x=arr[0]
    self.y=arr[1]
    self.z=arr[2]
    self.invkine()
    
# Inverse Kinematics
  def invkine(self):
    if (self.x==0 and self.y==0) :
      print("Singularity !!!!!!!")
      return
    else:
      self.theta1 = np.arctan2(self.y,self.x)

    r= (self.x**2+self.y**2)**(0.5)
    s=self.z-self.a1

    self.theta2 = np.arctan2(s,r)+(np.pi/2)

    self.d3=(r**2+s**2)**(0.5) - self.a2
# Forward Kinematics
  def forward(self):
    x1=(self.a2+self.d3)*np.cos(self.theta1)
    y1=(self.a2+self.d3)*np.sin(self.theta1)
    z1=self.a1+(self.a2+self.d3)*np.sin(self.theta2)

    return [x1,y1,z1]

  def verify(self,arr):
    self.setpos(arr)
    print(f"Original Co-ordinates:- {[self.x,self.y,self.z]}\n")
    print(f"Verified from code :- {self.forward()}\n")
    print(f"Joint parameters: \n theta1: {self.theta1} rad \n theta2: {self.theta2} rad \n d3: {self.d3} m \n\n")


# SCARA Configuration
Implementation of Inverse and Forward Kinematics

In [47]:
class scara():
  def __init__(self):
    self.x=0
    self.y=0
    self.z=0

    self.theta1=0
    self.theta2=0
    self.d3=0

    self.a1=0.25
    self.a2=0.25
    self.h=0.25
  
  def setpos(self,arr):
    self.x=arr[0]
    self.y=arr[1]
    self.z=arr[2]
    self.invkine()
    
# Inverse Kinematics
  def invkine(self):
    r= ((self.x**2+self.y**2-self.a1**2-self.a2**2)/(2*self.a1*self.a2))
    self.theta2 = np.arctan2(((1-r**2)**0.5),r)
    self.theta1 = np.arctan2(self.y,self.x)-np.arctan2(self.a2*np.sin(self.theta2),(self.a1+self.a2*np.cos(self.theta2)))

    self.d3=self.h-self.z
# Forward Kinematics
  def forward(self):
    x1=self.a1*np.cos(self.theta1)+self.a2*np.cos(self.theta1+self.theta2)
    y1=self.a1*np.sin(self.theta1)+self.a2*np.sin(self.theta1+self.theta2)
    z1=self.h-self.d3;

    return [x1,y1,z1]

  def verify(self,arr):
    self.setpos(arr)
    print(f"Original Co-ordinates:- {[self.x,self.y,self.z]}\n")
    print(f"Verified from code :- {self.forward()}\n")
    print(f"Joint parameters: \n theta1: {self.theta1} rad \n theta2: {self.theta2} rad \n d3: {self.d3} m \n\n")

# Inputs

In [48]:
lett = ["A","B","C","D"]
cord = [[0.4,0.06,0.1],[0.4,0.01,0.1],[0.35,0.01,0.1],[0.35,0.06,0.1]]

# Verification

**Puma**

In [49]:
puma1 = puma()
for i in range(len(cord)):
  print(lett[i])
  puma1.verify(cord[i])

A
Original Co-ordinates:- [0.4, 0.06, 0.1]

Verified from code :- [0.39999999999999997, 0.05999999999999999, 0.10000000000000003]

Joint parameters: 
 theta1: 0.14888994760949723 rad 
 theta2: 0.1749129587876629 rad 
 theta3: -1.0600826270863724 rad 


B
Original Co-ordinates:- [0.4, 0.01, 0.1]

Verified from code :- [0.40000000000000013, 0.010000000000000004, 0.09999999999999998]

Joint parameters: 
 theta1: 0.02499479361892016 rad 
 theta2: 0.18728199258035133 rad 
 theta3: -1.091899934609392 rad 


C
Original Co-ordinates:- [0.35, 0.01, 0.1]

Verified from code :- [0.35, 0.01, 0.10000000000000003]

Joint parameters: 
 theta1: 0.028563657838759998 rad 
 theta2: 0.29990350570505653 rad 
 theta3: -1.4092951796178457 rad 


D
Original Co-ordinates:- [0.35, 0.06, 0.1]

Verified from code :- [0.35, 0.05999999999999999, 0.09999999999999998]

Joint parameters: 
 theta1: 0.16977827396833844 rad 
 theta2: 0.29075369217343694 rad 
 theta3: -1.3808563007082255 rad 




**Stanford**

In [50]:
stf=stanford()
for i in range(len(cord)):
  print(lett[i])
  stf.verify(cord[i])

A
Original Co-ordinates:- [0.4, 0.06, 0.1]

Verified from code :- [0.42662031125527783, 0.06399304668829167, 0.6544749683231337]

Joint parameters: 
 theta1: 0.14888994760949723 rad 
 theta2: 1.2156679720393733 rad 
 d3: 0.18139309220245986 m 


B
Original Co-ordinates:- [0.4, 0.01, 0.1]

Verified from code :- [0.4271837383182708, 0.010679593457956769, 0.6501249804748512]

Joint parameters: 
 theta1: 0.02499479361892016 rad 
 theta2: 1.212128352070552 rad 
 d3: 0.17731721238443 m 


C
Original Co-ordinates:- [0.35, 0.01, 0.1]

Verified from code :- [0.38076455669427, 0.010878987334122002, 0.6001428280002319]

Joint parameters: 
 theta1: 0.028563657838759998 rad 
 theta2: 1.1660522426910302 rad 
 d3: 0.13091993909481814 m 


D
Original Co-ordinates:- [0.35, 0.06, 0.1]

Verified from code :- [0.3799442757255618, 0.06513330441009631, 0.605105618091294]

Joint parameters: 
 theta1: 0.16977827396833844 rad 
 theta2: 1.1711218686142209 rad 
 d3: 0.13548670534792762 m 




**SCARA**

In [51]:
scr=scara()
for i in range(len(cord)):
  print(lett[i])
  scr.verify(cord[i])

A
Original Co-ordinates:- [0.4, 0.06, 0.1]

Verified from code :- [0.4, 0.06, 0.1]

Joint parameters: 
 theta1: -0.4795426595751926 rad 
 theta2: 1.2568652143693797 rad 
 d3: 0.15 m 


B
Original Co-ordinates:- [0.4, 0.01, 0.1]

Verified from code :- [0.4, 0.010000000000000009, 0.1]

Joint parameters: 
 theta1: -0.6180895978105604 rad 
 theta2: 1.2861687828589612 rad 
 d3: 0.15 m 


C
Original Co-ordinates:- [0.35, 0.01, 0.1]

Verified from code :- [0.35, 0.010000000000000009, 0.1]

Joint parameters: 
 theta1: -0.7664350954805546 rad 
 theta2: 1.5899975066386292 rad 
 d3: 0.15 m 


D
Original Co-ordinates:- [0.35, 0.06, 0.1]

Verified from code :- [0.35, 0.06, 0.1]

Joint parameters: 
 theta1: -0.6112198326377974 rad 
 theta2: 1.5619962132122718 rad 
 d3: 0.15 m 


